In [ ]:
from jupyter_dash import JupyterDash
import numpy as np
import pandas as pd
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import dash_html_components as html
from dash.dependencies import Input, Output, State

# Remove warning slice assignment using loc
pd.set_option('mode.chained_assignment', None)

# Load Data
df = pd.read_excel('cancer_mama.xlsx', squeeze=True)
df['age_at_diagnosis'] = np.round(df['age_at_diagnosis'])
# Clear columns that will not be used
df = df.drop(df.columns[range(520, 693)], axis=1)
df.rename(columns={"pam50_+_claudin-low_subtype": "Molecular_subtype"},  inplace = True)

# App declaration
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LUX])

# Define values for multiple dropdowns
multiAxisColumnNames = [
    {'value': 'age_at_diagnosis', 'label': 'Age at Diagnosis'},
    {'value': 'type_of_breast_surgery', 'label': 'Type of Breast Surgery'},
    {'value': 'cancer_type_detailed', 'label': 'Type of Breast Cancer'},
    {'value': 'chemotherapy', 'label': 'Chemotherapy'},
    {'value': 'death_from_cancer', 'label': 'Cause of Death'},
]
cancer_type_detailed = [cancer for cancer in df['cancer_type_detailed'].unique()]
Molecular_subtype = [classification for classification in df['Molecular_subtype'].unique()]

# Top navbar
navbar = dbc.Navbar([
    html.A(
        dbc.Row(
            [
                dbc.Col(html.Img(src=app.get_asset_url("logo.png"), height="40px"), width=3),
                dbc.Col(dbc.NavbarBrand("OncoView"), width=9, align="center"),
            ],
            align="center",
            no_gutters=True,
        ),
    )
], color="lightblue")

# Specific layout, contains the scatter plot and heatmap of the application
specific_layout = html.Div([
    html.Br(),
    html.Br(),
    html.H4("Factors Involved in Breast Cancer Patient Outcome"),
    html.Br(),
    html.Br(),
    dbc.Row(
        [
            dbc.Col(html.Div([
                dbc.Card(
                    dbc.CardBody(
                        [
                            html.H4('Filter Panel'),
                            html.Br(),
                            html.H5('Scatter Plot Filters'),
                            html.Br(),
                            html.Br(),
                            html.H5('Chemotherapy'),
                            dcc.Checklist(id="chemotherapy_filterYN",
                                          options=[
                                              {'label': 'Yes', 'value': 1},
                                              {'label': 'No', 'value': 0}
                                          ],
                                          value=[0, 1],
                                          labelStyle=dict(display='block')
                                          ),

                            html.Br(),
                            html.H5('Radiotherapy'),
                            dcc.Checklist(id="Radiotherapy_filterYN",
                                          options=[
                                              {'label': 'Yes', 'value': 1},
                                              {'label': 'No', 'value': 0}
                                          ],
                                          value=[0, 1],
                                          labelStyle=dict(display='block')
                                          ),

                            html.Br(),
                            html.H5('Hormone Therapy'),
                            dcc.Checklist(id="Hormonetherapy_filterYN",
                                          options=[
                                              {'label': 'Yes', 'value': 1},
                                              {'label': 'No', 'value': 0}
                                          ],
                                          value=[0, 1],
                                          labelStyle=dict(display='block')
                                          ),
                            html.Br(),
                            html.H5('Cause of death'),
                            dcc.Checklist(id="survival_filterYN",
                                          options=[
                                               {'label': 'Living', 'value': 'Living'},
                                               {'label': 'Died of Disease', 'value': 'Died of Disease'},
                                               {'label': 'Died of Other Causes', 'value': 'Died of Other Causes'}
                                              ],
                                          value=['Living','Died of Disease','Died of Other Causes'],
                                          labelStyle=dict(display='block')
                                          ),
                            html.Br(),
                            html.H4('General Filters'),
                            html.Br(),
                            html.H5('Age at Diagnosis'),
                            dcc.RangeSlider(id="agerange_filter",
                                            marks={i: str(i) for i in range(20, 101, 10)},
                                            min=20,
                                            max=100,
                                            value=[22, 96]
                                            ),
                            html.Br(),
                            html.H5('Molecular Classification'),
                            dcc.Dropdown(id="classification",
                                         options=[
                                             {'label': i, 'value': i} for i in df.Molecular_subtype.unique()],
                                         value= Molecular_subtype,
                                         multi=True,
                                         placeholder="Select Molecular Classification"
                                         ),
                            html.Br(),
                            html.H5('Select Cancer Type'),
                            dcc.Dropdown(id="cancer_type",
                                         options=[
                                             {'label': i, 'value': i} for i in df.cancer_type_detailed.unique()],
                                         value=cancer_type_detailed,
                                         multi=True,
                                         placeholder="Select Cancer Type"
                                         ),
                        ]))]), width=3),

            dbc.Col(html.Div([

                html.Br(),

                html.H5('Overall Survival of Patients over Age at Diagnosis per Molecular Type'),
                dcc.Graph(id='scatterplot', figure={}),

                html.Br(),

                html.H5('Expression of Oncogenes and Tumor Suppresor Genes over Cancer Type'),
                html.Br(),
                dcc.Graph(id='htmp', figure={}),

            ])),
        ]
    )

], style={"marginLeft": "10px", "marginRight": "10px"})

# General layout, allows the user to use the multi axis graph
general_layout = html.Div([
    html.Br(),
    html.Br(),
    html.H4("Factors Involved in Breast Cancer Patient Outcome"),
    html.Br(),
    html.Br(),
    dbc.Row([
        dbc.Col(html.Div([
            dbc.Card(
                dbc.CardBody(
                    [
                        # Filters that affect all the dataset
                        html.H4("Filter Panel"),
                        html.Br(),
                        html.Div([

                            # Age filter, using a longer range than what the data has cause it's easier
                            # to show it on the UI
                            html.H5('Age at Diagnosis'),
                            dcc.RangeSlider(
                                id="age_filter",
                                marks={i: str(i) for i in range(20, 101, 10)},
                                min=20,
                                max=100,
                                value=[22, 96]
                            ),

                            # Chemotherapy filter, binary options, yes or no
                            html.Br(),
                            html.H5('Chemotherapy'),
                            dcc.Checklist(
                                id="chemotherapy_filter",
                                options=[
                                    {'label': ' Yes', 'value': 1},
                                    {'label': ' No', 'value': 0}
                                ],
                                value=[0, 1],
                                labelStyle=dict(display='block')
                            ),

                            html.Br(),
                            html.H5('Cause of death'),
                            dcc.Checklist(id="survival_filter",
                                          options=[
                                               {'label': 'Living', 'value': 'Living'},
                                               {'label': 'Died of Disease', 'value': 'Died of Disease'},
                                               {'label': 'Died of Other Causes', 'value': 'Died of Other Causes'}
                                              ],
                                          value=['Living','Died of Disease','Died of Other Causes'],
                                          labelStyle=dict(display='block')
                                          ),

                            # Cancer type filter, getting the data from the dataframe
                            html.Br(),
                            html.H5('Select Cancer Type'),
                            dcc.Dropdown(
                                id="cancer_type_filter",
                                options=[{'label': cancer, 'value': cancer} for cancer in cancer_type_detailed],
                                value=cancer_type_detailed,
                                multi=True,
                                placeholder="Select Cancer Type"
                            )

                        ]),
                    ]
                )
            )
        ]), width=3),
        dbc.Col(html.Div([
            # Axis selection dropdown
            html.H5('Select an Axis to Visualize the Number of Patients'),
            html.Br(),
            dcc.Dropdown(
                id="multi_axis_selection",
                options=multiAxisColumnNames,
                multi=False,
                value="age_at_diagnosis",
                style={"width": "75%"},
                placeholder="Select an Option"
            ),
            html.Br(),

            html.H5(id='multi_axis_label', children=['Multi Axis Graph']),
            html.Br(),
            dcc.Graph(id='multi_axis_graph')
        ]), width=9),
    ]
    ),

], style={"marginLeft": "10px", "marginRight": "10px"})

# Survival layout, allows the user to see survival data by conditions
survival_layout = html.Div([
    html.Br(),
    html.Br(),
    html.H4("Treatments and Survival"),   
    html.Br(),
    html.Br(),
    dbc.Row(
        [
            dbc.Col(html.Div([
                dbc.Card(
                    dbc.CardBody(
                        [
                            html.H4('Filter Panel'),
                            html.Br(),
                            html.H5('Age at Diagnosis'),
                            dcc.RangeSlider(id="agerange_filter2",
                                            marks={i: str(i) for i in range(20, 101, 10)},
                                            min=20,
                                            max=100,
                                            value=[22, 96]
                                            ),

                            html.Br(),
                            html.H5('Chemotherapy'),
                            dcc.Checklist(id="chemotherapy_filterYN2",
                                          options=[
                                              {'label': ' Yes', 'value': 1},
                                              {'label': ' No', 'value': 0}
                                          ],
                                          value=[0, 1],
                                          labelStyle=dict(display='block')
                                          ),

                            html.Br(),
                            html.H5('Radiotherapy'),
                            dcc.Checklist(id="Radiotherapy_filterYN2",
                                          options=[
                                              {'label': ' Yes', 'value': 1},
                                              {'label': ' No', 'value': 0}
                                          ],
                                          value=[0, 1],
                                          labelStyle=dict(display='block')
                                          ),

                            html.Br(),
                            html.H5('Hormone Therapy'),
                            dcc.Checklist(id="Hormonetherapy_filterYN2",
                                          options=[
                                              {'label': ' Yes', 'value': 1},
                                              {'label': ' No', 'value': 0}
                                          ],
                                          value=[0, 1],
                                          labelStyle=dict(display='block')
                                          ),

                            html.Br(),
                            html.H5('ER status'),
                            dcc.Checklist(id="er_status_filterYN2",
                                          options=[
                                              {'label': ' Positive', 'value': 'Positive'},
                                              {'label': ' Negative', 'value': 'Negative'}
                                          ],
                                          value=['Positive', 'Negative'],
                                          labelStyle=dict(display='block')
                                          ),

                           html.Br(),
                            html.H5('Cause of death'),
                            dcc.Checklist(id="survival_filterYN2",
                                          options=[
                                               {'label': 'Living', 'value': 'Living'},
                                               {'label': 'Died of Disease', 'value': 'Died of Disease'},
                                               {'label': 'Died of Other Causes', 'value': 'Died of Other Causes'}
                                              ],
                                          value=['Living','Died of Disease','Died of Other Causes'],
                                          labelStyle=dict(display='block')
                                          ),

                            html.Br(),
                            html.H5('Select Cancer Type'),
                            dcc.Dropdown(id="cancer_type2",
                                         options=[
                                             {'label': i, 'value': i} for i in df.cancer_type_detailed.unique()],
                                         value=cancer_type_detailed,
                                         multi=True,
                                         placeholder="Select Cancer Type"
                                         ),

                        ]))]), width=3),

            dbc.Col(html.Div([

                html.Br(),

                html.H5('Survival of Patients per tumor size'),    
                dcc.Graph(id='survival_patients_graph'),

                html.Br(),
                html.H5('Survival of Patients per treatment'),    
                dcc.Graph(id='survival_treatment'),


            ])),
        ]
    )

], style={"marginLeft": "10px", "marginRight": "10px"})

# Creating main layout with all of the tabs
main_layout = html.Div([
    navbar,
    html.Br(),

    dbc.Tabs([
        dbc.Tab(general_layout, label="General Graphs"),
        dbc.Tab(specific_layout, label="Specific Graphs"),
        dbc.Tab(survival_layout, label="Survival Graphs")
    ]),
    html.Footer(html.Img(src=app.get_asset_url("acib.png"), height="150px"))
])

# Attach layout to application
app.layout = main_layout


# -------------------------------------------------CALLBACKS-----------------------------------------------------------#


# Callback to create the multi axis graph
@app.callback(
    [
        Output(component_id='multi_axis_graph', component_property='figure'),
        Output(component_id='multi_axis_label', component_property='children')
    ],
    [
        Input(component_id='age_filter', component_property='value'),
        Input(component_id='chemotherapy_filter', component_property='value'),
        Input(component_id='survival_filter', component_property='value'),
        Input(component_id='cancer_type_filter', component_property='value'),
        Input(component_id='multi_axis_selection', component_property='value')
    ],
    State("multi_axis_selection", "options")
)
def update_multi_axis_graph(
        age_range_filter,
        chemotherapy_filter,
        survival_filter,
        cancer_type_filter,
        column_selected,
        column_options
):
    graph_title = "Multiple Axis Graph"
    if column_selected is None:
        column_selected = "age_at_diagnosis"
    column_selected_label = ""
    for option in column_options:
        if option['value'] == column_selected:
            column_selected_label = option['label']
            graph_title = "Count of patients per {}".format(column_selected_label)
            if column_selected == "age_at_diagnosis":
                column_selected_label += " (Years)"

    dff = df.copy()
    dff = apply_age_range_filter(dff, age_range_filter)
    dff = apply_chemotherapy_filter(dff, chemotherapy_filter)
    dff = apply_survival_filterYN(dff, survival_filter)
    dff = apply_cancer_type_filter(dff, cancer_type_filter)

    if dff.empty:
        dff = df.copy()

    dff['count'] = 1
    dff = dff[[column_selected, 'count']].groupby([column_selected]).sum().reset_index()
    labels={
        column_selected: column_selected_label,
        "count": "Number of Patients"
    }

    if column_selected == "cancer_type_detailed" or column_selected == "death_from_cancer":
        return px.pie(
            dff, 
            values='count', 
            names=column_selected,
            labels=labels,
            color_discrete_sequence=px.colors.sequential.Blues[::-1]
        ), graph_title
    else:
        return px.bar(
            dff,
            x=column_selected,
            y="count",
            labels=labels,
            color=column_selected,
            color_discrete_sequence=px.colors.sequential.Blues[::-1][0:9:3],
            color_continuous_scale=px.colors.sequential.Blues,

        ), graph_title

#Callback scatterplot y heatmap------------------------------------------------------------------------------------------------

@app.callback(
    [
        Output(component_id='scatterplot', component_property='figure'),
        Output(component_id='htmp', component_property='figure')
    ],
    [
        Input(component_id='agerange_filter', component_property='value'),
        Input(component_id='chemotherapy_filterYN', component_property='value'),
        Input(component_id='Radiotherapy_filterYN', component_property='value'),
        Input(component_id='Hormonetherapy_filterYN', component_property='value'),
        Input(component_id='survival_filterYN', component_property='value'),
        Input(component_id='cancer_type', component_property='value'),
        Input(component_id='classification', component_property='value')
    ]
)
def update_scatter_heatmap(
        age_filter,
        chemotherapy_filter,
        radiotherapy_filter,
        hormone_therapy_filter,
        survival_filterYN,
        cancer_type,
        classification
):
    scatter_plot_dataframe = df.copy()
    heatmap_dataframe = df.copy()

    scatter_plot_dataframe = apply_age_range_filter(scatter_plot_dataframe, age_filter)
    scatter_plot_dataframe = apply_chemotherapy_filter(scatter_plot_dataframe, chemotherapy_filter)
    scatter_plot_dataframe = apply_radiotherapy_filter(scatter_plot_dataframe, radiotherapy_filter)
    scatter_plot_dataframe = apply_hormone_therapy_filter(scatter_plot_dataframe, hormone_therapy_filter)
    scatter_plot_dataframe = apply_survival_filterYN(scatter_plot_dataframe, survival_filterYN)
    scatter_plot_dataframe = apply_cancer_type_filter(scatter_plot_dataframe, cancer_type)
    scatter_plot_dataframe = apply_classification(scatter_plot_dataframe, classification)

    heatmap_dataframe = apply_age_range_filter(heatmap_dataframe, age_filter)
    heatmap_dataframe = apply_cancer_type_filter(heatmap_dataframe, cancer_type)
    heatmap_dataframe = apply_classification(heatmap_dataframe, classification)
   

    # Scatter plot sin el overlap y con el color by de grado de neoplasia y el symbol de celularidad
    scatter_plot_dataframe['Overall_survival_in_years'] = (scatter_plot_dataframe['overall_survival_months'] / 12)
    scatter_plot_dataframe.chemotherapy.loc[scatter_plot_dataframe.chemotherapy == 1] = 'Yes'
    scatter_plot_dataframe.chemotherapy.loc[scatter_plot_dataframe.chemotherapy == 0] = 'No'
    scatter_plot_dataframe.hormone_therapy.loc[scatter_plot_dataframe.hormone_therapy == 1] = 'Yes'
    scatter_plot_dataframe.hormone_therapy.loc[scatter_plot_dataframe.hormone_therapy == 0] = 'No'
    scatter_plot_dataframe.radio_therapy.loc[scatter_plot_dataframe.radio_therapy == 1] = 'Yes'
    scatter_plot_dataframe.radio_therapy.loc[scatter_plot_dataframe.radio_therapy == 0] = 'No'
    scatter_plot_dataframe['neoplasm_histologic_grade'] = scatter_plot_dataframe['neoplasm_histologic_grade'] \
        .astype('str')
    
    if not scatter_plot_dataframe.empty:
        fig = px.scatter(
            scatter_plot_dataframe,
            x='Overall_survival_in_years',
            y="age_at_diagnosis",
            color='Molecular_subtype',
            labels={'Overall_survival_in_years': 'Overall Survival (Years)',
                    "age_at_diagnosis": 'Age at Diagnosis',
                    'Molecular_subtype': 'Molecular Subtype', 'chemotherapy': 'Chemotherapy',
                    'hormone_therapy': 'Hormone Therapy', 'type_of_breast_surgery': 'Type of Breast Surgery',
                    'radio_therapy': 'Radio Therapy', "cellularity": "Cellularity"},
            hover_data=['chemotherapy', 'hormone_therapy', 'type_of_breast_surgery', 'radio_therapy'],
            color_continuous_scale="sunset",
            symbol="cellularity"
        )

    else:
        fig = {}
        
        
    # Heatmap
    heatmap_dataframe = heatmap_dataframe[
        ['cancer_type_detailed','tp53', 'myc','pten', 'pik3ca', 'brca1', 'brca2','chek2', 'hras','kras', 'nras',
         'akt2', 'stat3', 'stat2', 'stat5a', 'erbb2', 'erbb3','gata3', 'stk11']
    ]

    heatmap_unpivoted = heatmap_dataframe.melt(
        id_vars=['cancer_type_detailed'],
        var_name='Oncogenes',
        value_name='Results'
    )

    heatmap_group_by = heatmap_unpivoted.groupby(['cancer_type_detailed', 'Oncogenes'])[['Results']]\
        .mean().reset_index()
    dff = heatmap_group_by.pivot(index='cancer_type_detailed', columns='Oncogenes', values='Results')

    if not dff.empty:
        heatmap = px.imshow(
            dff,
            labels=dict(
                x="Tumor Suppressor Genes & Oncogenes",
                y="Cancer Type Detailed",
                color="Expression Levels"
            ),
            x=['tp53', 'stk11' ,'myc','pten', 'brca1', 'brca2','chek2', 'hras','kras', 'nras',
         'akt2', 'stat3', 'stat2', 'stat5a', 'pik3ca', 'erbb2', 'erbb3', 'gata3'],
            color_continuous_scale="RdBu"
        )
        heatmap.update_xaxes(side="top")
    else:
        heatmap = {}

    return fig, heatmap


@app.callback([Output(component_id='survival_patients_graph', component_property='figure'),
              Output(component_id='survival_treatment', component_property='figure')
              ],
              [Input(component_id='agerange_filter2', component_property='value'),
               Input(component_id='chemotherapy_filterYN2', component_property='value'),
               Input(component_id='Radiotherapy_filterYN2', component_property='value'),
               Input(component_id='Hormonetherapy_filterYN2', component_property='value'),
               Input(component_id='er_status_filterYN2', component_property='value'),
                Input(component_id='survival_filterYN2', component_property='value'),
               Input(component_id='cancer_type2', component_property='value')]
              )
def update_survival_patients_graph(
        age_filter,
        chemotherapy_filter,
        radiotherapy_filter,
        hormone_therapy_filter,
        er_status_filter,
        survival_filterYN,
        cancer_type
):
    dff1 = df.copy()
    dff2 = df.copy()
    dff1 = apply_age_range_filter(dff1, age_filter)
    dff1 = apply_chemotherapy_filter(dff1, chemotherapy_filter)
    dff1 = apply_radiotherapy_filter(dff1, radiotherapy_filter)
    dff1 = apply_hormone_therapy_filter(dff1, hormone_therapy_filter)
    dff1 = apply_er_status_filter(dff1, er_status_filter)
    dff1 = apply_survival_filterYN(dff1, survival_filterYN)
    dff1 = apply_cancer_type_filter(dff1, cancer_type)
    dff1['Overall_survival_in_years'] = (dff1['overall_survival_months'] / 12)    
    
    if not dff1.empty:
        fig = px.density_contour(
        dff1,
        x="tumor_size",
        y="Overall_survival_in_years",
                labels={
            "tumor_size": "Tumor size (mm)",
            "Overall_survival_in_years": "Overall Survival (Years)" 
                })
        fig.update_traces(contours_coloring="fill",
                      contours_showlabels=True,
                      colorscale="Blues", 
                      colorbar= dict(title="Count of patients",
                                    titleside="top"))
    else:
        fig = {}
        
    dff2 = apply_age_range_filter(dff2, age_filter)
    dff2 = apply_chemotherapy_filter(dff2, chemotherapy_filter)
    dff2 = apply_radiotherapy_filter(dff2, radiotherapy_filter)
    dff2 = apply_hormone_therapy_filter(dff2, hormone_therapy_filter)
    dff2 = apply_er_status_filter(dff2, er_status_filter)
    dff2 = apply_survival_filterYN(dff2, survival_filterYN)
    dff2 = apply_cancer_type_filter(dff2, cancer_type)

    if not dff2.empty:
        fig2 = px.histogram(dff2, 
        x="type_of_breast_surgery", 
        color="chemotherapy", 
        barmode="group", 
        labels={
            "type_of_breast_surgery": "Type of breast surgery",
            "chemotherapy": "Chemotherapy",
            "hormone_therapy": "Hormone therapy",
            "radio_therapy":"Radio therapy",
            "count": "Count of Patients"
        },
        color_discrete_sequence=px.colors.sequential.Blues[::-1][1:6:3],
        facet_row="hormone_therapy", 
        facet_col="radio_therapy"
                           )
        
        fig2.update_yaxes(showline=True, linewidth=2, linecolor='black', title="Count of patients")
        
    else:
        fig2 = {}
        
    return fig, fig2

# --------------------------------------------------FILTERS------------------------------------------------------------#


# Function to apply filter by cancer type
def apply_cancer_type_filter(dataset, cancer_type_selected):
    if bool(cancer_type_selected):
        return dataset[dataset['cancer_type_detailed'].isin(cancer_type_selected)]
    return dataset


# Function to apply filter by age
def apply_age_range_filter(dataset, age_range):
    if bool(age_range):
        return dataset[dataset['age_at_diagnosis'].isin(range(age_range[0], age_range[1] + 1))]
    return dataset


# Function to apply filter by chemotherapy
def apply_chemotherapy_filter(dataset, chemotherapy):
    if bool(chemotherapy):
        temp_dataset = dataset[dataset['chemotherapy'].isin(chemotherapy)]
        temp_dataset.loc[temp_dataset['chemotherapy'] == 0, 'chemotherapy'] = "No"
        temp_dataset.loc[temp_dataset['chemotherapy'] == 1, 'chemotherapy'] = "Yes"
        return temp_dataset
    return dataset


# Function to apply filter by survival
def apply_survival_filter(dataset, survival):
    if bool(survival):
        temp_dataset = dataset[dataset['overall_survival'].isin(survival)]
        temp_dataset.loc[temp_dataset['overall_survival'] == 0, 'overall_survival'] = "No"
        temp_dataset.loc[temp_dataset['overall_survival'] == 1, 'overall_survival'] = "Yes"
        return temp_dataset
    return dataset


# Function to apply filter by radiotherapy
def apply_radiotherapy_filter(dataset, radio_therapy_selected):
    if bool(radio_therapy_selected):
        temp_dataset = dataset[dataset['radio_therapy'].isin(radio_therapy_selected)]
        temp_dataset.loc[temp_dataset['radio_therapy'] == 0, 'radio_therapy'] = "No"
        temp_dataset.loc[temp_dataset['radio_therapy'] == 1, 'radio_therapy'] = "Yes"
        return temp_dataset
    return dataset


# Function to apply filter by hormone therapy
def apply_hormone_therapy_filter(dataset, hormone_therapy_selected):
    if bool(hormone_therapy_selected):
        temp_dataset = dataset[dataset['hormone_therapy'].isin(hormone_therapy_selected)]
        temp_dataset.loc[temp_dataset['hormone_therapy'] == 0, 'hormone_therapy'] = "No"
        temp_dataset.loc[temp_dataset['hormone_therapy'] == 1, 'hormone_therapy'] = "Yes"
        return temp_dataset
    return dataset


# Function to apply filter by er_status
def apply_er_status_filter(dataset, er_status_selected):
    if bool(er_status_selected):
        temp_dataset = dataset[dataset['er_status'].isin(er_status_selected)]
        temp_dataset.loc[temp_dataset['er_status'] == 'Negative', 'er_status'] = "Negative"
        temp_dataset.loc[temp_dataset['er_status'] == 'Positive', 'er_status'] = "Positive"
        return temp_dataset
    return dataset

# Function to apply filter by cause of death
def apply_survival_filterYN(dataset, death_from_cancer):
    if bool(death_from_cancer):
        return dataset[dataset['death_from_cancer'].isin(death_from_cancer)]
    return dataset

# Function to apply filter by classification
def apply_classification(dataset, Molecular_subtype):
    if bool(Molecular_subtype):
        return dataset[dataset['Molecular_subtype'].isin(Molecular_subtype)]
    return dataset

# ---------------------------------------------------------------------------------------------------------------------#


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, port=8050)
